In [30]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [31]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6933, 7)

In [32]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6914, 5)

In [33]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6939, 9)

fin empresas y comienzo de rescate de pacientes


In [34]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1629548, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [35]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1195587, 4)

In [36]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1195587, 12)

In [38]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [39]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [40]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [41]:
#pacientes_empresas22.shape

In [42]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [43]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [44]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [48]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife Agrosuper"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [49]:
Pacientes_Archivo.head(10)

,Holding,Empresa,Rut Titular,Dv Titular,Unnamed: 4,Rut Asegurado,Dv,rut1,sexo,Parentesco,nombres,apellidos,Fecha Nacimiento,Edad,email,xcvcxvxc,Unnamed: 16,Unnamed: 17
0,Agrosuper,AGROSUPER COM. ALIM. LTDA,10000760,6,10000760-6,10000760,6,10000760-6,F,TITULAR,FABIOLA DEL C,POZO DELGADO,1965-02-02,57,FABIPOZO@GMAIL.COM,340011776.0,340011776.0,MetLife
1,Agrosuper,AGROSUPER COM. ALIM. LTDA,10000760,6,10000760-6,9068895,2,9068895-2,M,CÓNYUGE,EDUARDO,CONTRERAS PARRA,1965-09-12,57,FABIPOZO@GMAIL.COM,340011776.0,340011776.0,MetLife
2,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,10002540,K,10002540-K,M,TITULAR,LUIS HUMBERTO,CARO MADRID,1966-11-22,56,caroluis054@gmail.com,340011805.0,340011805.0,MetLife
3,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,12693854,3,12693854-3,F,CÓNYUGE,GIOVANNA,JIMENEZ ROJAS,1974-10-29,48,caroluis054@gmail.com,340011805.0,340011805.0,MetLife
4,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,20787902,9,20787902-9,F,HIJO,NICOL ANDREA,CARO JIMENEZ,2001-07-16,21,caroluis054@gmail.com,340011805.0,340011805.0,MetLife
5,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,21810446,0,21810446-0,F,HIJO,CATHERINE GIOVANNA,CARO JIMENEZ,2005-04-05,17,caroluis054@gmail.com,340011805.0,340011805.0,MetLife
6,Agrosuper,FAENADORA LO MIRANDA LTDA,10005989,4,10005989-4,10005989,4,10005989-4,M,TITULAR,JUAN ALBERTO,CAMPOS SANDOVAL,1964-05-21,58,juancampos.ssa@gmail.com,340011804.0,340011804.0,MetLife
7,Agrosuper,AGRICOLA SUPER LTDA,10009826,1,10009826-1,10009826,1,10009826-1,M,TITULAR,FELIPE EDUARDO,GALVEZ CATALAN,1966-06-08,56,maestrofgalvez@gmail.com,340011811.0,340011811.0,MetLife
8,Agrosuper,AGRICOLA SUPER LTDA,10009826,1,10009826-1,10375520,4,10375520-4,F,CÓNYUGE,JOSEFINA DEL CARME,DIAZ PEREZ,1966-06-10,56,maestrofgalvez@gmail.com,340011811.0,340011811.0,MetLife
9,Agrosuper,SOPRAVAL PRODUCCIÓN,10010792,9,10010792-9,10010792,9,10010792-9,M,TITULAR,ELADIO EDMUNDO,ALVARADO GUTIERREZ,1966-08-19,56,alvarado.gutierrez.eladio@gmail.com,340011941.0,340011941.0,MetLife


In [50]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [51]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Holding,Empresa,Rut Titular,Dv Titular,Unnamed: 4,Rut Asegurado,Dv,rut1,sexo,Parentesco,nombres,apellidos,Fecha Nacimiento,Edad,email,xcvcxvxc,Unnamed: 16,Unnamed: 17,rut
0,Agrosuper,AGROSUPER COM. ALIM. LTDA,10000760,6,10000760-6,10000760,6,10000760-6,F,TITULAR,FABIOLA DEL C,POZO DELGADO,1965-02-02,57,FABIPOZO@GMAIL.COM,340011776.0,340011776.0,MetLife,100007606
1,Agrosuper,AGROSUPER COM. ALIM. LTDA,10000760,6,10000760-6,9068895,2,9068895-2,M,CÓNYUGE,EDUARDO,CONTRERAS PARRA,1965-09-12,57,FABIPOZO@GMAIL.COM,340011776.0,340011776.0,MetLife,90688952
2,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,10002540,K,10002540-K,M,TITULAR,LUIS HUMBERTO,CARO MADRID,1966-11-22,56,caroluis054@gmail.com,340011805.0,340011805.0,MetLife,10002540K
3,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,12693854,3,12693854-3,F,CÓNYUGE,GIOVANNA,JIMENEZ ROJAS,1974-10-29,48,caroluis054@gmail.com,340011805.0,340011805.0,MetLife,126938543
4,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,20787902,9,20787902-9,F,HIJO,NICOL ANDREA,CARO JIMENEZ,2001-07-16,21,caroluis054@gmail.com,340011805.0,340011805.0,MetLife,207879029


In [52]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_8972\485331713.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Holding,Empresa,Rut Titular,Dv Titular,Unnamed: 4,Rut Asegurado,Dv,rut1,sexo,Parentesco,nombres,apellidos,Fecha Nacimiento,Edad,email,xcvcxvxc,Unnamed: 16,Unnamed: 17,rut
0,Agrosuper,AGROSUPER COM. ALIM. LTDA,10000760,6,10000760-6,10000760,6,10000760-6,F,TITULAR,FABIOLA DEL C,POZO DELGADO,1965-02-02,57,FABIPOZO@GMAIL.COM,340011776.0,340011776.0,MetLife,100007606
1,Agrosuper,AGROSUPER COM. ALIM. LTDA,10000760,6,10000760-6,9068895,2,9068895-2,M,CÓNYUGE,EDUARDO,CONTRERAS PARRA,1965-09-12,57,FABIPOZO@GMAIL.COM,340011776.0,340011776.0,MetLife,90688952
2,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,10002540,K,10002540-K,M,TITULAR,LUIS HUMBERTO,CARO MADRID,1966-11-22,56,caroluis054@gmail.com,340011805.0,340011805.0,MetLife,10002540K
3,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,12693854,3,12693854-3,F,CÓNYUGE,GIOVANNA,JIMENEZ ROJAS,1974-10-29,48,caroluis054@gmail.com,340011805.0,340011805.0,MetLife,126938543
4,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,20787902,9,20787902-9,F,HIJO,NICOL ANDREA,CARO JIMENEZ,2001-07-16,21,caroluis054@gmail.com,340011805.0,340011805.0,MetLife,207879029


In [53]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_8972\2118328728.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_8972\2118328728.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [54]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(26440, 29)

In [55]:
Pacientes_con_id.head()

,Holding,Empresa,Rut Titular,Dv Titular,Unnamed: 4,Rut Asegurado,Dv,rut1,sexo,Parentesco,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,Agrosuper,AGROSUPER COM. ALIM. LTDA,10000760,6,10000760-6,10000760,6,10000760-6,F,TITULAR,...,True,0018c00002axqM8AAI,100007606,Pozo Delgado Fabiola,RUT,None,None,None,Hombre,None
1,Agrosuper,AGROSUPER COM. ALIM. LTDA,10000760,6,10000760-6,9068895,2,9068895-2,M,CÓNYUGE,...,True,0018c00002axqM9AAI,90688952,Contreras Parra Eduardo,RUT,None,None,None,Hombre,None
2,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,10002540,K,10002540-K,M,TITULAR,...,True,0018c00002ayp8gAAA,10002540K,Luis Humberto Caro Madrid,RUT,None,None,None,Hombre,None
3,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,12693854,3,12693854-3,F,CÓNYUGE,...,True,0018c00002ayuI3AAI,126938543,Giovanna Jimenez Rojas,RUT,None,None,None,Hombre,None
4,Agrosuper,PROC.DE ALIMENTOS DEL SUR,10002540,K,10002540-K,20787902,9,20787902-9,F,HIJO,...,True,0018c00002ayxwyAAA,207879029,Nicol Andrea Caro Jimenez,RUT,None,None,None,Hombre,None


In [56]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"].astype(str).str.upper()
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"].apply(str).str.strip().replace("+","")
    Subir_pacientes['Phone'] = Subir_pacientes['Phone'].str.title().replace("NaN", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtsEAE'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,HealthCloudGA__Gender__pc,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [57]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "45",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [58]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
600336,a1Y8c00000Doa5ZEAR,0018c00002kCg8nAAC,a1W8c0000083sB9EAI,MetLife Agrosuper,0018c00002gjmIHAAY,Proc. De Alimentos Del Sur,Agrosuper 2,MetLife Agrosuper,Proc. De Alimentos Del Sur,11805,EMP10219,A-9403
602888,a1Y8c00000Doaw5EAB,0018c00002kCMf4AAG,a1W8c0000083sB5EAI,MetLife Agrosuper,0018c00002gjmIDAAY,Agrosuper S.A.,Agrosuper 2,MetLife Agrosuper,Agrosuper S.A.,11810,EMP10215,A-9399
1115917,a1Y8c00000Dz6TDEAZ,0018c00002axqM8AAI,a1W8c0000083sB4EAI,MetLife Agrosuper,0018c00002gjmICAAY,Agrosuper Com. Alim. Ltda.,Agrosuper 2,MetLife Agrosuper,Agrosuper Com. Alim. Ltda.,11802,EMP10214,A-9398
1115918,a1Y8c00000Dz6TEEAZ,0018c00002ayp8gAAA,a1W8c0000083sB9EAI,MetLife Agrosuper,0018c00002gjmIHAAY,Proc. De Alimentos Del Sur,Agrosuper 2,MetLife Agrosuper,Proc. De Alimentos Del Sur,11805,EMP10219,A-9403
1115919,a1Y8c00000Dz6TFEAZ,0018c00002axhqiAAA,a1W8c0000083sB4EAI,MetLife Agrosuper,0018c00002gjmICAAY,Agrosuper Com. Alim. Ltda.,Agrosuper 2,MetLife Agrosuper,Agrosuper Com. Alim. Ltda.,11802,EMP10214,A-9398


In [59]:
campaña_seleccionada.shape

(26002, 12)

In [60]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [61]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [62]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
